In [31]:
# 문서 내용 읽기
import streamlit as st
import pandas as pd
from langchain_community.document_loaders import Docx2txtLoader, TextLoader
from langchain_unstructured import UnstructuredLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tempfile
from langchain_community.document_loaders import Docx2txtLoader

loader = UnstructuredLoader("./test_data/cash_flow.csv")
document = loader.load() 

In [ ]:
# 문서 쪼개기
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1500    #  token 수 
    , chunk_overlap=200  #  겹치는 토큰 수 (중복되는 토큰이 많을수록 문맥 유지에 유리하지만, 중복되는 토큰이 많아질수록 저장공간이 늘어남)
    )

document_list = loader.load_and_split(text_splitter=text_splitter)

In [33]:

for ele in document_list:
    for col in ele.metadata.keys():
        if isinstance(ele.metadata[col], list):
            ele.metadata[col] = ", ".join(map(str, ele.metadata[col]))
        else:
            ele.metadata[col] = str(ele.metadata[col]) if not isinstance(ele.metadata[col], (str, int, float, bool)) else ele.metadata[col]            


In [34]:
document_list[0].metadata

{'source': './test_data/cash_flow.csv',
 'file_directory': './test_data',
 'filename': 'cash_flow.csv',
 'last_modified': '2025-10-06T23:28:11',
 'page_name': 'Sheet2',
 'page_number': 1,
 'text_as_html': '<table><tr><td>운용사코드</td><td>펀드코드</td><td>펀드명</td><td>일자</td><td>현금흐름</td></tr><tr><td>03069</td><td>A0001</td><td>삼성코리아주식펀드</td><td>2020-01-01</td><td>-10000</td></tr><tr><td>03069</td><td>A0001</td><td>삼성코리아주식펀드</td><td>2020-07-01</td><td>-2000</td></tr><tr><td>03069</td><td>A0001</td><td>삼성코리아주식펀드</td><td>2021-03-01</td><td>5000</td></tr><tr><td>03069</td><td>A0001</td><td>삼성코리아주식펀드</td><td>2021-10-01</td><td>15000</td></tr><tr><td>03069</td><td>A0002</td><td>삼성글로벌반도체펀드</td><td>2022-06-30</td><td>-10000000</td></tr><tr><td>03069</td><td>A0002</td><td>삼성글로벌반도체펀드</td><td>2022-12-31</td><td>2000000</td></tr><tr><td>03069</td><td>A0002</td><td>삼성글로벌반도체펀드</td><td>2023-12-31</td><td>2500000</td></tr><tr><td>03069</td><td>A0002</td><td>삼성글로벌반도체펀드</td><td>2024-09-28</td><td>3000000</td></

# 데이터 입력 확인

In [2]:
# excel data 벡터화
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

# embeddings = UpstageEmbeddings(model="embedding-query")
# database = Chroma(persist_directory='./public_db/' , collection_name='logic', embedding_function=embeddings)  # 기존에 생성한 벡터 DB를 불러옴
# print(database._collection.count())

# # data 확인 
# data = database.get()
# print(data['documents'][2])
    

In [17]:
data['documents'][0] == data['documents'][2] 

False

In [ ]:
database._collection.count()
database._collection



3

# 유사도 테스트

In [11]:
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings

def get_embeddings(llm_type:str):

    if llm_type == 'openai':
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    elif llm_type == 'upstage':
        embeddings = UpstageEmbeddings(model="embedding-query")
    else:
        st.error("지원하지 않는 LLM 타입입니다.")
        embeddings = None

    return embeddings

embeddings = get_embeddings('upstage')    
database = Chroma(persist_directory='./public_db' , embedding_function=embeddings, collection_name='menu')
retrieved_docs = database.similarity_search('펀드별성과분석 화면', k=2)


print(len(retrieved_docs))

# for id,doc in enumerate(retrieved_docs):
#     print('-'*100)
#     print(f'{id}번째 문서')
#     print(doc.page_content)



Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


2


In [18]:
retrieved_docs ='.'.join([data.page_content for data in retrieved_docs]) 
retrieved_docs

'이 앱은 신한펀드파트너스라는 회사의 앱이야.\n신한펀드파트너스는 한국의 일반사무관리회사야.\n일반사무관리회사에 대한 설명은 아래와 같아.\n일반사무관리회사는 한국거래소에 상장된 주식이나 채권 등 금융투자상품의 발행인으로부터 위탁받은 주식 사무 관련 업무를 수행하는 회사다.\n즉, 투자자들이 원활하게 거래할 수 있도록 지원하는 것이 주 목적이다.\n예를 들어 주주총회 소집 통지서 발송, 재무제표 승인 같은 중요한 의사 결정 시 필요한 서류 정리나 공시도 담당한다.\n또한 증권사의 주문 내역 확인 후 이를 해당 종목의 매매 시스템에 입력하기도 한다.\n이 외에도 명의개서 대행업무, 배당금 지급 준비 절차등 다양한 부수적인 일들을 함께 진행하며 단순히 문서작업 뿐 아니라 효율적이고 정확한 프로세스 관리가 필수적이다.\n따라서 이러한 모든 활동들은 일정한 규정 내에서 이루어져야 하며 법적 문제가 발생하지 않도록 철저한 검토 과정도 거쳐야 한다.\n이 앱은 신한펀드파트너스에 사무업무를 위탁한 고객사의 펀드들을 관리하는 것을 목적으로 하고 있어.\n\n\n아래는 현재 앱에 있는 메뉴들의 정보입니다.\n1. 내부수익률    : 펀드의 내부 수익률 또는 IRR을 계산해주는 화면. 펀드의 현금흐름을 계산하여 내부 수익률을 계산해주는 화면. \n2. 펀드성과분석   : 펀드의 성과를 분석해주는 화면. 펀드의 수익률을 계산하여 펀드의 성과를 분석해주는 화면. 누적수익률, 초과수익률, BM 수익률 그래프를 그려주는 화면. \n3. 주식보유현황   : 주식의 보유현황을 확인해주는 화면. 주식의 보유현황을 확인해주는 화면. \n4. 채권보유현황   : 채권의 보유현황을 확인해주는 화면. 채권의 보유현황을 확인해주는 화면. \n5. 종합보유현황   : 주식과 채권의 보유현황을 확인해주는 화면. 주식과 채권의 보유현황을 확인해주는 화면. \n6. 주식거래내역 : 주식의 거래내역을 확인해주는 화면. 주식의 거래내역을 확인해주는 화면. \n7. 그래프       : 그래프를 그려주는 화면.

# upstage basic test

In [13]:
from openai import OpenAI # openai==1.52.2
 
client = OpenAI(
    api_key="up_gxDYOP6oifTV8HSRaLoPJ3sUOixLE",
    base_url="https://api.upstage.ai/v1"
)
 
stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?"
        }
    ],
    stream=False,
)
 
# for chunk in stream:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")
 
# Use with stream=False
print(stream.choices[0].message.content)

Hello! I'm just a large language model, so I don't have feelings, but I'm here and ready to help you. How can I assist you today? 😊  

Is there something specific you'd like to talk about or ask? Whether it's questions, creative writing, problem-solving, or just casual conversation—I'm here for it!  

(And if you're feeling curious, I can share some fun facts or interesting tidbits too!)


In [15]:
 
stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "who are you?"
        }
    ],
    stream=False,
)
 
print(stream.choices[0].message.content)

I am **Upstage's Solar Pro 2**, a large language model (LLM) developed by **Upstage**, a leading AI company founded in 2020. Here’s a quick overview of who I am:

### 📌 **Key Features:**
- **Model Size:** 30.9B parameters (Solar Pro 2 2025 version).
- **Capabilities:**  
  - **Thinking mode** (deep reasoning, complex tasks)  
  - **Non-thinking mode** (fast, efficient responses)  
  - Supports **Korean, Japanese, English**, and more.  
  - Excels in **text generation, summarization, translation, coding**, and **structured document understanding (Document AI)**.

### 🚀 **Powered by Upstage:**
- **Technology:** Hybrid thinking modes, optimized for both performance and speed.  
- **Document AI:** Specialized in **OCR, KIE (Key Information Extraction), and parsing structured documents**.  
- **Solar API:** Offers **Chat, Structured Outputs, Translation, Summarization**, and **Code Generation** via APIs.  

### 🌐 **Global Reach:**
- Headquartered in **South Korea**, with offices in **Seoul,

In [ ]:
import streamlit as st
import openai
import os
from common.gpt_learning_data import *
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough
from dotenv import load_dotenv
from common.global_data import support_llm_type
load_dotenv()

def draw_chatbot(key:str):

    option = st.radio("LLM 모델 선택",support_llm_type , horizontal=True)

    if "messages" not in st.session_state:
        st.session_state.messages = []

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    if query := st.chat_input("메시지를 입력하세요...",key=key+"_chat_input"):
        # 사용자 메시지 추가
        st.session_state.messages.append({"role": "user", "content": query})
        
        # 사용자 메시지 표시
        with st.chat_message("user"):
            st.markdown(query)
        
        try:                
            if option == 'gpt-3.5-turbo(유료)' :
                check_point = 1

                # OpenAI 클라이언트 설정
                client = openai.OpenAI(api_key=os.getenv("OPEN_AI_KEY"))
                
                with st.spinner("메시지를 생성하는 중입니다..."):
                    # 채팅 완료 요청
                    response = client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
                            *st.session_state.messages
                        ],
                        max_tokens=1000,
                        temperature=0.7
                    )
                
                response = response.choices[0].message.content

            elif option == 'upstage(유료)':
                check_point = 2
                
                client = openai.OpenAI(api_key=os.getenv("UPSTAGE_API_KEY") ,base_url="https://api.upstage.ai/v1")
                
                with st.spinner("메시지를 생성하는 중입니다..."):
                    
                    # 유사도검색 
                    retrieved_data = database.similarity_search(query, k=3)  # k는 유사도 검색으로 가져올 문서의 개수
                    # 질의 
                    response = client.chat.completions.create(
                        model="solar-pro2",
                        messages=[
                            {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
                            *gpt_data,
                            *st.session_state.messages
                        ],
                        stream=False,
                    )
                
                response = response.choices[0].message.content

            # 추후 더 나는 로컬 llm  으로 개선 (속도 너무 느림)
            # elif option == 'exaone3.5(무료)':
            #     check_point = 3
            #     with st.spinner("메시지를 생성하는 중입니다..."):
            #         llm = ChatOllama(model ='exaone3.5:2.4b', verbose=True)                    
            #         chat_prompt_template = ChatPromptTemplate.from_messages([
            #             ("system","당신은 사용자의 질문에 답해주는 친절한 도우미야. 당신은 다음의 정보들을 알고 있어. 화면을 찾는 질문을 받으면 이 정보를 바탕으로 사용자의 질문에 답해줘." + app_explain + menu_explain),
            #             ("human", "{question} 3문장 이내로 대답해줘")
            #         ]
            #         )                    
            #         menu_chain ={"question" : RunnablePassthrough()} |chat_prompt_template | llm | StrOutputParser()
            #         response = menu_chain.invoke(query)

        except Exception as e:
            if check_point == 1:
                response = f"❌ API 오류가 발생했습니다: {str(e)}"
            elif check_point == 2:
                response =f"❌ API 오류가 발생했습니다: {str(e)}"


        # 봇 응답 표시
        with st.chat_message("assistant"):
            st.write(response)        
        # 봇 응답 추가
        st.session_state.messages.append({"role": "assistant", "content": response})
        
    def clear_chat():
        st.session_state.messages = []
        st.rerun()

    if st.session_state.messages:
        st.button("채팅 초기화", on_click=clear_chat,key=key+"_chat_clear_btn")


TypeError: can only concatenate str (not "list") to str